## Importing Libraries

In [1]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np

## Electric Cars

In [2]:
r = requests.get("https://www.zigwheels.com/newcars/electric-cars")
soup = BeautifulSoup(r.text, 'lxml')
links = []
for i in soup.find_all("a", class_="fnt-14 lnk-hvr of-hid ht-name"):
    links.append(i.attrs['href'])

In [3]:
def car_features(links):
    key_car_feat = {}
    r = requests.get(links)
    soup_cars = BeautifulSoup(r.text, 'lxml')
    for i in soup_cars.find_all('script', type="application/ld+json")[-1]:
        data = json.loads(i)
        key_car_feat['Vehicle_type'] = data.get('@type')
        key_car_feat['Make'] = data.get('brand').get('name')
        key_car_feat['BodyType'] = data.get('bodyType')
        key_car_feat['Model'] = data.get('name')
        key_car_feat['Starting_Price'] = np.round(data.get('offers').get('price')/100000,2)
        key_car_feat['seatingCapacity'] = data.get('seatingCapacity')
    for i in data.get('additionalProperty'):
        key_car_feat[i.get('name')] = i.get('value')
    return key_car_feat

In [4]:
df_cars = pd.DataFrame()
for i in links:
    feats = car_features(i)
    df2_cars = pd.DataFrame([feats])
    df_cars = pd.concat([df_cars, df2_cars], axis=0)
df_cars.reset_index(drop=True, inplace=True)
df_cars.head()

,Vehicle_type,Make,BodyType,Model,Starting_Price,seatingCapacity,Max Torque,Max Power,Range,Battery Capacity,Charging Time,Ground Clearance,Boot Space,Fast Charging
0,Car,Tata,SUV,Tata Punch EV,10.99,5,190Nm,120.69 - 80.46bhp,315 - 421 km/charge,25 - 35kwh,50Mins,190 mm,366 Liters,NaN
1,Car,Tata,SUV,Tata Nexon EV,14.74,5,215Nm,127.39 - 142.68bhp,325 - 465 km,30 - 40.5 kwh,6H -AC-7.2 kW (10-100%),190 mm,350 Liters,Yes
2,Car,Kia,SUV,Kia EV6,60.95,5,605Nm,225.86 - 320.55bhp,708 km,77.4 kwh,18 Min (0-80%),NaN,NaN,Yes
3,Car,Tata,Hatchback,Tata Tiago EV,8.69,5,114Nm,60.34 - 73.75bhp,250 - 315 km/charge,19.2 - 24 kwh,58 min| DC-25 kW(10-80%),NaN,240 Liters,Yes
4,Car,Mahindra,SUV,Mahindra XUV400 EV,15.49,5,310Nm,147.51 - 149.55bhp,375 - 456 kms,34.5 - 39.4 kwh,6H 30 Min-AC-7.2 kW (0-100%),NaN,NaN,Yes


In [5]:
cols = {"Vehicle_type":"VehicleType",
        "Make":"Make",
        "BodyType":"BodyType",
        "Model":"Model",
        "Starting_Price":"StartingPrice(INR Lakhs)"   ,
        "seatingCapacity":"NoofSeats",
        "Max Torque":"MaxTorque(Nm)",
        "Max Power":"MaxPower(BHP)",
        "Range":"Range(km/charge)",
        "Battery Capacity":"BatteryCapacity(kwh)",
        "Charging Time":"ChargingTime(Mins)",
        "Ground Clearance":"GroundClearance(mm)",
        "Boot Space":"BootSpace(Liters)",
        "Fast Charging":"FastCharging"
}
df_cars = df_cars.rename(columns=cols)

In [7]:
df_cars.to_csv("Electric_cars_vehicle_spec.csv", index=None)

## Electric Bikes

In [8]:
r = requests.get("https://www.zigwheels.com/newbikes/electric-bikes")
soup = BeautifulSoup(r.text, 'lxml')
links = []
for i in soup.find_all("a", class_="fnt-14 lnk-hvr of-hid ht-name"):
    links.append(i.attrs['href'])

In [9]:
def bike_features(links):
    key_bike_feat = {}
    r = requests.get(links)
    soup_bike = BeautifulSoup(r.text, 'lxml')
    for i in soup_bike.find_all('script', type="application/ld+json")[-1]:
        data = json.loads(i)
        key_bike_feat['Vehicle_type'] = "Bikes"
        key_bike_feat['Make'] = data.get('brand').get('name')
        key_bike_feat['Model'] = data.get('name')
        key_bike_feat['Starting_Price'] = np.round(data.get('offers').get('lowPrice')/100000,2)
    for i in data.get('additionalProperty'):
        key_bike_feat[i.get('name')] = i.get('value')
    return key_bike_feat

In [11]:
df_bikes = pd.DataFrame()
for i in links:
    feats = bike_features(i)
    df2_bikes = pd.DataFrame([feats])
    df_bikes = pd.concat([df_bikes, df2_bikes], axis=0)
df_bikes.reset_index(drop=True, inplace=True)
df_bikes.head()

,Vehicle_type,Make,Model,Starting_Price,Range,Battery Capacity,Charging Time,Top Speed,Kerb Weight,Tyre Type,Ground Clearance,Fast Charging
0,Bikes,Revolt Motors,Revolt RV400,1.17,150 - 80-150 km/charge,3.24 Kwh,4.5 Hr,85 km/Hr,108 Kg,Tubeless,215 mm,NaN
1,Bikes,Ultraviolette,Ultraviolette F77,3.80,206 - 307 km/charge,10.3 Kwh,4 Hr,152 km/Hr,207 Kg,Tubeless,160 mm,Yes
2,Bikes,Tork,Tork Kratos,1.67,180 km/charge,NaN,6-7 Hr,105 km/Hr,140 Kg,Tubeless,165 mm,No
3,Bikes,Oben,Oben Rorr,1.50,200 km/charge,NaN,2 Hr,100 km/Hr,NaN,Tubeless,230 mm,Yes
4,Bikes,Komaki,Komaki Ranger,1.86,200-250 km/charge,NaN,6 Hr,NaN,NaN,Tubeless,NaN,NaN


In [12]:
df_bikes.columns

Index(['Vehicle_type', 'Make', 'Model', 'Starting_Price', 'Range',
       'Battery Capacity', 'Charging Time', 'Top Speed', 'Kerb Weight',
       'Tyre Type', 'Ground Clearance', 'Fast Charging'],
      dtype='object')

In [ ]:
['Vehicle_type', 'Make', 'Model', 'Starting_Price', 'Range',
       'Battery Capacity', 'Charging Time', 'Top Speed', 'Kerb Weight',
       'Tyre Type', 'Ground Clearance', 'Fast Charging']

['Vehicle_type', 'Make', 'Model', 'Starting_Price', 'Range',
       'Battery Capacity', 'Charging Time', 'Top Speed', 'Kerb Weight',
       'Tyre Type', 'Fast Charging']

## Electric Scooters

In [13]:
r = requests.get("https://www.zigwheels.com/newbikes/electric-scooters")
soup = BeautifulSoup(r.text, 'lxml')
links = []
for i in soup.find_all("a", class_="fnt-14 lnk-hvr of-hid ht-name"):
    links.append(i.attrs['href'])

In [14]:
def scooter_features(links):
    key_scoo_feat = {}
    r = requests.get(links)
    soup_scoo = BeautifulSoup(r.text, 'lxml')
    for i in soup_scoo.find_all('script', type="application/ld+json")[-1]:
        data = json.loads(i)
        key_scoo_feat['Vehicle_type'] = "Scooters"
        key_scoo_feat['Make'] = data.get('brand').get('name')
        key_scoo_feat['Model'] = data.get('name')
        key_scoo_feat['Starting_Price'] = np.round(data.get('offers').get('lowPrice')/100000,2)
    for i in data.get('additionalProperty'):
        key_scoo_feat[i.get('name')] = i.get('value')
    return key_scoo_feat

In [15]:
df_scoo = pd.DataFrame()
for i in links:
    feats = scooter_features(i)
    df2_scoo = pd.DataFrame([feats])
    df_scoo = pd.concat([df_scoo, df2_scoo], axis=0)
df_scoo.reset_index(drop=True, inplace=True)
df_scoo.head()

,Vehicle_type,Make,Model,Starting_Price,Range,Battery Capacity,Charging Time,Top Speed,Kerb Weight,Tyre Type,Ground Clearance,Fast Charging
0,Scooters,Ola Electric,Ola S1 Pro,1.40,181 - 195 km/charge,4 Kwh,6.5 Hr,120 km/Hr,125 Kg,Tubeless,165 mm,Yes
1,Scooters,TVS,TVS iQube Electric,1.17,100 - 145 km/charge,3.4 Kwh,NaN,78 km/Hr,118.8 Kg,Tubeless,157 mm,Yes
2,Scooters,Bajaj,Bajaj Chetak,1.15,113 - 127 km/charge,3.2 Kwh,4.3 Hr,73 km/Hr,134 Kg,Tubeless,160 mm,NaN
3,Scooters,Ather Energy,Ather 450X,1.26,111 - 150 km/charge,3.7 Kwh,5.4 Hr,90 km/Hr,111.6 Kg,Tubeless,170 mm,Yes
4,Scooters,Vida,Vida V1 Pro,1.46,110 km/charge,3.94 Kwh,6 Hr,80 km/Hr,125 Kg,Tubeless,155 mm,Yes


In [16]:
df_scoo.columns

Index(['Vehicle_type', 'Make', 'Model', 'Starting_Price', 'Range',
       'Battery Capacity', 'Charging Time', 'Top Speed', 'Kerb Weight',
       'Tyre Type', 'Ground Clearance', 'Fast Charging'],
      dtype='object')

In [17]:
df_two_wheelers = pd.concat([df_bikes,df_scoo], axis=0)
df_two_wheelers.head()

,Vehicle_type,Make,Model,Starting_Price,Range,Battery Capacity,Charging Time,Top Speed,Kerb Weight,Tyre Type,Ground Clearance,Fast Charging
0,Bikes,Revolt Motors,Revolt RV400,1.17,150 - 80-150 km/charge,3.24 Kwh,4.5 Hr,85 km/Hr,108 Kg,Tubeless,215 mm,NaN
1,Bikes,Ultraviolette,Ultraviolette F77,3.80,206 - 307 km/charge,10.3 Kwh,4 Hr,152 km/Hr,207 Kg,Tubeless,160 mm,Yes
2,Bikes,Tork,Tork Kratos,1.67,180 km/charge,NaN,6-7 Hr,105 km/Hr,140 Kg,Tubeless,165 mm,No
3,Bikes,Oben,Oben Rorr,1.50,200 km/charge,NaN,2 Hr,100 km/Hr,NaN,Tubeless,230 mm,Yes
4,Bikes,Komaki,Komaki Ranger,1.86,200-250 km/charge,NaN,6 Hr,NaN,NaN,Tubeless,NaN,NaN


In [20]:
cols = {"Vehicle_type":"VehicleType",
        "Make":"Make",
        "Model":"Model",
        "Starting_Price":"StartingPrice(INR Lakhs)",
        "Range":"Range(km/charge)",
        "Battery Capacity":"BatteryCapacity(kwh)",
        "Charging Time":"ChargingTime(Mins)",
        "Top Speed":"Top Speed(km/Hr)",
        "Kerb Weight":"KerbWeight(Kg)",
        "Tyre Type":"TyreType",
        "Ground Clearance":"GroundClearance(mm)",
        "Fast Charging":"FastCharging"
}
df_two_wheelers = df_two_wheelers.rename(columns=cols)
df_two_wheelers.head()

,VehicleType,Make,Model,StartingPrice(INR Lakhs),Range(km/charge),BatteryCapacity(kwh),ChargingTime(Mins),Top Speed(km/Hr),KerbWeight(Kg),TyreType,GroundClearance(mm),FastCharging
0,Bikes,Revolt Motors,Revolt RV400,1.17,150 - 80-150 km/charge,3.24 Kwh,4.5 Hr,85 km/Hr,108 Kg,Tubeless,215 mm,NaN
1,Bikes,Ultraviolette,Ultraviolette F77,3.80,206 - 307 km/charge,10.3 Kwh,4 Hr,152 km/Hr,207 Kg,Tubeless,160 mm,Yes
2,Bikes,Tork,Tork Kratos,1.67,180 km/charge,NaN,6-7 Hr,105 km/Hr,140 Kg,Tubeless,165 mm,No
3,Bikes,Oben,Oben Rorr,1.50,200 km/charge,NaN,2 Hr,100 km/Hr,NaN,Tubeless,230 mm,Yes
4,Bikes,Komaki,Komaki Ranger,1.86,200-250 km/charge,NaN,6 Hr,NaN,NaN,Tubeless,NaN,NaN


In [21]:
df_two_wheelers.to_csv('electric_two_wheeler_specs.csv', index=None)